In [ ]:
# Install required libraries
!pip install transformers

In [ ]:
pip uninstall pandas


Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pandas-2.2.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pandas/*
Proceed (Y/n)? 

In [ ]:
pip install pandas

In [ ]:
pip install --upgrade pandas


In [ ]:
import pandas as pd


In [ ]:
# Load your dataset
df = pd.read_csv("path/to/your_dataset.csv")

# Combine relevant information into a single text entry for fine-tuning
df["combined_text"] = df["name"] + " is priced at " + df["UnitPrice"].astype(str) + " on " + df["InvoiceDate"]

# Save the preprocessed dataset with the combined text
df.to_csv("path/to/your_processed_dataset.txt", index=False)

# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"])

dataset = TextDataset(tokenizer=tokenizer, file_path="path/to/your_processed_dataset.txt", block_size=128, overwrite_cache=True)

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir="./fine-tuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2
)

# Create Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the model
trainer.save_model("./fine-tuned_model")

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./fine-tuned_model")
tokenizer = GPT2Tokenizer.from_pretrained("./fine-tuned_model")

# Function to generate descriptions based on input product names
def generate_description(product_name):
    input_text = product_name + " is priced at"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Test the model with an input product name
input_product_name = "ZOHO CRM"
description = generate_description(input_product_name)
print("Description for", input_product_name + ":", description)

False


AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)